In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#np.show_config() use threads?

def update_data(sub_pops, max_receptors):
    #rows that says how many bacteria have nonperm
    #columns that say how many bacteria have perm
    #this will be MxM sized matrix
    #flatten this matrix into array?
    # -- > row i and col j --> number bacteria with
    data = np.zeros((max_receptors,max_receptors),dtype=np.int32)
    sub_pop_keys = list(sub_pops.keys())

    for i in sub_pop_keys:
        pop = sub_pops[i]
        receptors = pop[:,-1] #the last column of pop is number of receptors
        #0 == free receptors
        #1 == nonperm receptors
        #2 == perm receptors
        #3 == number of receptors for that bacteria
        species, count = np.unique(pop[:,1:3], axis = 0, return_counts = True)
        for s in enumerate(species):
            spc = s[1]
            i = int(spc[0]) #nonperm bounded
            j = int(spc[1]) #perm bounded
            data[i,j] = count[s[0]]
    return data;

#suppose m receptors
#suppose x bacteria
#suppose y phages
#each bacteria may have n phages bound to it
#each bacteria may have n' phages permanently bound to it
#n+n'<=m

#initialize an array containing all the available bacteria and their receptors
def initiate_bacteria(num_bacteria, receptors):
    #different rows represent different bacteria
    #different columns represent receptor freedom
    #3 states of receptor freedom
    #0 == free
    #1 == nonperm bounded
    #2 == perm bounded
    #The sum of these must be less than or equal to m
    #m must be less than or equal to M
    M = receptors
    #each bacteria can have a stochastic number of m receptors less than or equal to M
    #A healthy, non phage challenged bacteria may have up to 150+ receptors
    m_rows = np.random.randint(low = 50, high = 150, size = (num_bacteria,1),dtype=np.int32) #generates random number of receptors
    free_m = m_rows
    non_perm_m = np.zeros((num_bacteria,1), dtype = np.int32)
    perm_m = np.zeros((num_bacteria,1), dtype=np.int32)

    pop = np.hstack((free_m,non_perm_m))
    pop = np.hstack((pop,perm_m))
    pop = np.hstack((pop,m_rows))
    #the last column of pop will tell you the number of receptors for each bacteria
    #when we update m, it must not be lower than the sum of the first 3 columns of pop for each bacteria
    return pop;

#for large numbers of bacteria, we must divide population into subpops
def define_pop(num_bacteria, receptors):
    #divide bacteria population into subpops, each size 100,000
    num_bacteria = int(num_bacteria)
    n = num_bacteria/100000
    n = int(n)
    sub_pops = {}
    for i in range(n):
        sub_pops[i] = initiate_bacteria(num_bacteria, receptors)

    return sub_pops;

def prob_matrix(P0,rx1,rx2,rx3,N,k1,k2,k3,MOI,pop):
    reactions = np.stack((rx1,rx2,rx3),axis=1)
    P = P0*reactions
    p_choose1 = P[:,0]/np.nansum(P[:,0])
    p_choose2 = P[:,1]/np.nansum(P[:,1])
    p_choose3 = P[:,2]/np.nansum(P[:,2])
    choices_i = {}
    #randomly choose from 0 to N for each reaction if reactions are possible:
    if np.nansum(P[:,0]) != 0:
        chooser1 = np.random.choice(list(range(0,N)),p = p_choose1)
        #exception of impossible indices
        choices_i[0] = chooser1
    if np.nansum(P[:,1]) != 0:
        chooser2 = np.random.choice(list(range(0,N)),p = p_choose2)
        #exception of impossible indices
        choices_i[1] = chooser2
    if np.nansum(P[:,2]) != 0:
        chooser3 = np.random.choice(list(range(0,N)),p = p_choose3)
        #exception of impossible indices
        choices_i[2] = chooser3

    prob_ind = list(range(len(choices_i)))
    p_choose_f = []
    for u in prob_ind:
        p_choose_f.append(P[:,u][choices_i[u]])

    p_choose_f = np.array(p_choose_f)
    p_choose_f = p_choose_f/np.nansum(p_choose_f)
    print('prob rxn: {}'.format(p_choose_f))
    choosej = np.random.choice(prob_ind, p = p_choose_f)
    
    j_f = choosej  #which reaction on the bacteria will occur (r1,r2,r3)
    i_f = choices_i[choosej] #this is the row index of the selected bacteria
    bact = pop[i_f]
    if j_f == 0:
        w = MOI*k1*bact[0]
    if j_f == 1:
        w = k2*bact[1]
    if j_f == 2:
        w = k3*bact[2]
    return  [j_f, i_f, w];


def gillespie_reaction_weights(sub_pops,num_phages,tau,MOI,k1,k2,k3):
    #calculate P0

    candidates = np.zeros((len(sub_pops),3),dtype=np.float32) #sized 12000 max
    reactions = 0
    for i in range(len(sub_pops)):
        pop = sub_pops[i]
        N = pop.shape[0]
        rx1 = np.nansum(pop[:,0])*MOI*k1
        rx2 = np.nansum(pop[:,1])*k2
        rx3 = np.nansum(pop[:,1])*k3
        reactions += (rx1+rx2+rx3)

    del rx1, rx2, rx3, pop

    P0 = np.exp(-tau*reactions)

    for i in range(len(sub_pops)):
        pop = sub_pops[i]
        N = pop.shape[0]
        rx1 = pop[:,0]*MOI*k1
        rx2 = pop[:,1]*k2
        rx3 = pop[:,1]*k3
        candidates[i] = prob_matrix(P0,rx1,rx2,rx3,N,k1,k2,k3,MOI,pop)

    del rx1, rx2, rx3, pop
    #for each subpop, choose the a candidate bacteria
    #for each bacteria, choose the heaviest reaction on each
    #choose the heaviest bacteria
    weights = candidates[:,2]/np.nansum(candidates[:,2])
    print('subpop weights: {}'.format(weights))
    bacteria_ind = np.random.choice(range(0,len(weights)), p=weights)
    print('chosen subpop: {}'.format(bacteria_ind))
    #bacteria_ind is both the subpop index chosen, and location of values in candidates
    bacteria = candidates[bacteria_ind]
    pop = sub_pops[bacteria_ind]
    j_f = bacteria[0] #which reaction is occuring
    i_f = bacteria[1] #which bacteria in the chosen subpop
    i_f = int(i_f)
    bact = pop[i_f]
    print('rx: {}'.format(j_f))
    if j_f == 0:  #rx1 occurs
        bact[0]-=1
        bact[1]+=1
        num_phages-=1
    if j_f == 1:  #rx2 occurs
        bact[1]-=1
        bact[0]+=1
        num_phages+=1
    if j_f == 2:   #rx3 occurs
        bact[1]-=1
        bact[2]+=1
    #update
    pop[i_f] = bact
    sub_pops[bacteria_ind] = pop
    del bact, pop
    return sub_pops,num_phages;

def gillespie_time(pop,MOI,num_phages,k1,k2,k3): #returns a tau for each subpop
    #receives subpop...  3x100000 sized population matrix
    #find smallest tau for each subpop
    N = pop.shape[0]
    tau_j = np.random.uniform(size=(N,3))

    rx1 = pop[:,0]*num_phages*k1
    rx2 = pop[:,1]*k2
    rx3 = pop[:,2]*k3
    #we must decide which reactions are possible for a_u
    #find any impossible reactions and exclude them from analysis and choice
    #reactions that are impossible are those that have no reactants
    #thus au = hu*ku == 0
    #check rx1
    inclusion_r1 = np.where(rx1!=0)
    #check rx2
    inclusion_r2 = np.where(rx2!=0)
    #check rx3
    inclusion_r3 = np.where(rx3!=0)
    #each tau is weighted by its respective reaction for each bacteria
    tau_j[:,0] = (1/rx1) * np.log(1/(tau_j[:,0]))
    #issue:  if X molecules of species S == 0, then tau--> -inf
    tau_j[:,1] = (1/rx2) * np.log(1/tau_j[:,1])
    tau_j[:,2] = (1/rx3) * np.log(1/tau_j[:,2])
    #if tau == -inf --> np.nan
    tau_j[:,0][np.where(tau_j[:,0] == -np.inf)] = np.nan
    tau_j[:,1][np.where(tau_j[:,1] == -np.inf)] = np.nan
    tau_j[:,2][np.where(tau_j[:,2] == -np.inf)] = np.nan
    #only choose those that are not np.nan because they are impossible
    min_i = np.nanmin(tau_j,axis=1)  #set of all mins for each bacteria
    i = np.nanargmin(min_i) #min of all mins, the index of this is within N --> says what row it belongs to
    j = np.nanargmin(tau_j[i])
    #print(min_i)
    mintau = tau_j[i,j] #final tau for all calculations

    return mintau;


def gillespie(sub_pops,num_phages,k1,k2,k3):
    #k1 = assoicate
    #k2 = dissociate
    #k3 = perm

    #generate set of 3 taus for each bacteria based on receptor status
    #must do this for all subpops
    #define new dictionary for time respective to each sub_pops
    sub_pop_keys = list(sub_pops.keys())
    N = 0
    for t in sub_pop_keys:
        N+=list(sub_pops[t].shape)[0] #number of bacteria each iteration
        #in the future, bacteria may die and be unavailable
    MOI = num_phages/N
    phages = MOI

    tau = np.zeros((len(sub_pop_keys)))
    for t in sub_pop_keys:
        tau[t] = gillespie_time(sub_pops[t],MOI,num_phages,k1,k2,k3)
    tau = np.nanmin(tau) #find minimum tau from subpops
    #mutate pop
    sub_pops,num_phages = gillespie_reaction_weights(sub_pops,num_phages,tau,MOI,k1,k2,k3)
    return sub_pops,num_phages,tau;

def run_gillespie():
    tot_time = 10000
    num_bacteria = 1e+6
    num_phages = num_bacteria*5e-4
    k1,k2,k3 = 1.2e-11,8.5e-4,8e-4
    max_receptors = 100
    time = np.zeros((tot_time))
    data_all = np.zeros((tot_time,max_receptors,max_receptors),dtype=np.int32)
    free_phages = np.zeros((tot_time),dtype=np.int32)

    sub_pops = define_pop(num_bacteria, max_receptors)


    for t in range(tot_time):
        print('time step {}'.format(t))
        print('number phages: {}'.format(num_phages))
        sub_pops,num_phages,tau = gillespie(sub_pops,num_phages,k1,k2,k3)
        data_all[t] = update_data(sub_pops, max_receptors)
        free_phages[t] = num_phages
        time[t] = time[t-1]+tau
        print('time (s): {}'.format(time[t]))
        if num_phages == 0:
            break;
    return data_all,free_phages,time

data_all,free_phages,time = run_gillespie()


time step 0
number phages: 500.0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:198: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:199: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in true_divide


prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
subpop weights: [0.11798288 0.09514748 0.07802094 0.05233112 0.09609897 0.10085633
 0.13130353 0.06470029 0.13320647 0.13035205]
chosen subpop: 5
rx: 0.0
time (s): 0.06423483065471516
time step 1
number phages: 499.0
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.92341818e-11 5.15151515e-01 4.84848485e-01]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
subpop weights: [0.06952965 0.1002045  0.09406953 0.12474437 0.14417177 0.
 0.10224948 0.1002045  0.13599181 0.12883435]
chosen subpop: 8
rx: 0.0
time (s): 0.11745655423659024
time step 2
number phages: 498.0
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [4.09265455e-11 5.15151515e-01 4.84848485e-01]
prob rxn: [1.]
prob rxn: [1.]
prob rxn: [3.83912727e-11 5.15151515e-01 4.84848485e-01]
prob rxn: [1.

In [4]:

plt.plot(time,free_phages)
plt.plot(time,data_all[0:10000,0,1])
plt.show()


NameError: name 'time' is not defined